In [1]:
from fastapi import FastAPI, HTTPException, Path
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import pandas as pd
from typing import List, Dict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
#df_combinado2 = pd.merge(df_games2, df_items2, on='item_id',how='inner')

In [ ]:
listado_juegos = pd.read_parquet("df_funciones/listado_juegos_sin_repetir.parquet")

In [ ]:
df_combinado2 = pd.read_csv("df_funciones/df_recomend_juego_fc6.csv")

In [37]:
df_combinado2.head()

,genres,title,item_id,developer,playtime_forever
0,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,5.0
1,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,0.0
2,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,0.0
3,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,0.0
4,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,13.0


Esta funcion es muy lenta, por lo que vamos a limitarla

In [3]:
#@app.get('/RecomendacionJuego/{id_juego}')
def recomendacion_juego(id_juego: int):
    '''
    Endpoint para obtener una lista de juegos recomendados similares a un juego dado.

    Parámetros:
    - id_juego (int): ID del juego para el cual se desean obtener recomendaciones.

    Respuestas:
    - 200 OK: Retorna una lista con 5 juegos recomendados similares al juego ingresado.
    - 404 Not Found: Si no se encuentra el juego con el ID especificado.
    - 500 Internal Server Error: En caso de cualquier otro error, proporciona detalles de la excepción.

    Ejemplo de Uso:
    - /RecomendarJuego/123

    Ejemplo de Respuesta Exitosa:
    [
        {"id": 456, "nombre": "Juego A"},
        {"id": 789, "nombre": "Juego B"},
        {"id": 101, "nombre": "Juego C"},
        {"id": 202, "nombre": "Juego D"},
        {"id": 303, "nombre": "Juego E"}
    ]
    '''
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_combinado2[df_combinado2['item_id'] == id_juego]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

        title_game_and_genres = ' '.join(juego_seleccionado['title'].fillna('').astype(str) + ' ' + juego_seleccionado['genres'].fillna('').astype(str))
       
        tfidf_vectorizer = TfidfVectorizer()

        muestra_aleatoria = 10000
        ## similarity_scores = None

        chunk_tags_and_genres = df_combinado2['title'].fillna('').astype(str) + ' ' + df_combinado2['genres'].fillna('').astype(str)
        
        # Tomar una muestra aleatoria del conjunto de datos -- MDOIFICACION!!
        chunk_tags_and_genres_subset = chunk_tags_and_genres.sample(n=muestra_aleatoria)
        games_to_compare = [title_game_and_genres] + chunk_tags_and_genres_subset.tolist()


        tfidf_matrix = tfidf_vectorizer.fit_transform(games_to_compare)

        similarity_scores = cosine_similarity(tfidf_matrix)

        if similarity_scores is not None:
            similar_games_indices = similarity_scores[0].argsort()[::-1]

            num_recommendations = 5
            recommended_games = df_combinado2.loc[similar_games_indices[1:num_recommendations + 1]]

            return recommended_games[['title']].to_dict(orient='records')

        return {"message": "No se encontraron juegos similares."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}

In [20]:
def recomendacion_juego_lim(id_juego: int):
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_combinado2[df_combinado2['item_id'] == id_juego]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

        title_game_and_genres = ' '.join(juego_seleccionado['title'].fillna('').astype(str) + ' ' + juego_seleccionado['genres'].fillna('').astype(str))

        tfidf_vectorizer = TfidfVectorizer()

        muestra_aleatoria = 25000

        chunk_tags_and_genres = df_combinado2['title'].fillna('').astype(str) + ' ' + df_combinado2['genres'].fillna('').astype(str)

        # Tomar una muestra aleatoria del conjunto de datos
        chunk_tags_and_genres_subset = chunk_tags_and_genres.sample(n=muestra_aleatoria)
        games_to_compare = [title_game_and_genres] + chunk_tags_and_genres_subset.tolist()

        tfidf_matrix = tfidf_vectorizer.fit_transform(games_to_compare)

        similarity_scores = cosine_similarity(tfidf_matrix)

        if similarity_scores is not None:
            similar_games_indices = similarity_scores[0].argsort()[::-1]

            num_recommendations = 5
            recommended_games = df_combinado2.loc[similar_games_indices[1:num_recommendations + 1]]

            # Filtrar juegos duplicados
            recommended_games = recommended_games.drop_duplicates(subset=['title'])

            return recommended_games[['title']].to_dict(orient='records')

        return {"message": "No se encontraron juegos similares."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}


In [27]:
def recomendacion_juego_lim2(id_juego: int):
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_combinado2[df_combinado2['item_id'] == id_juego]
        
        # Ordenar el DataFrame por playtime_forever de mayor a menor
        juego_seleccionado = juego_seleccionado.sort_values(by='playtime_forever', ascending=False)

        # Obtener la primera fila de juego_seleccionado
        primera_fila = juego_seleccionado.iloc[[0]]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")
        
        
        title_game_and_genres = ' '.join(primera_fila['title'].fillna('').astype(str) + ' ' + primera_fila['genres'].fillna('').astype(str))

        tfidf_vectorizer = TfidfVectorizer()

        muestra_aleatoria = 15000

        chunk_tags_and_genres = df_combinado2['title'].fillna('').astype(str) + ' ' + df_combinado2['genres'].fillna('').astype(str)

        # Filtrar juegos duplicados antes de tomar una muestra aleatoria
        chunk_tags_and_genres = chunk_tags_and_genres.drop_duplicates(subset=['title'])

        # Tomar una muestra aleatoria del conjunto de datos
        chunk_tags_and_genres_subset = chunk_tags_and_genres.sample(n=muestra_aleatoria)
        games_to_compare = [title_game_and_genres] + chunk_tags_and_genres_subset.tolist()

        tfidf_matrix = tfidf_vectorizer.fit_transform(games_to_compare)

        similarity_scores = cosine_similarity(tfidf_matrix)

        if similarity_scores is not None:
            similar_games_indices = similarity_scores[0].argsort()[::-1]

            num_recommendations = 5
            recommended_games = df_combinado2.loc[similar_games_indices[1:num_recommendations + 1]]

            # Filtrar juegos duplicados
            #recommended_games = recommended_games.drop_duplicates(subset=['title'])

            return recommended_games[['title']].to_dict(orient='records')

        return {"message": "No se encontraron juegos similares."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}

In [51]:
def recomendacion_juego_lim3(id_juego: int):
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_combinado2[df_combinado2['item_id'] == id_juego]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

        # Obtener la primera fila de juego_seleccionado
        primera_fila = juego_seleccionado.iloc[[0]]

        title_game_and_genres = ' '.join(primera_fila['title'].fillna('').astype(str) + ' ' + primera_fila['genres'].fillna('').astype(str))

        tfidf_vectorizer = TfidfVectorizer()

        # Obtener las columnas 'title' y 'genres' antes de la concatenación
        chunk_tags_and_genres = df_combinado2[['title', 'genres']].fillna('').astype(str)

        # Agregar una nueva columna 'concatenated'
        chunk_tags_and_genres['concatenated'] = chunk_tags_and_genres['title'] + ' ' + chunk_tags_and_genres['genres']

        # Filtrar juegos duplicados antes de tomar una muestra aleatoria
        chunk_tags_and_genres = chunk_tags_and_genres.drop_duplicates(subset=['concatenated'])

        muestra_aleatoria = 25000

        # Tomar una muestra aleatoria del conjunto de datos, excluyendo los juegos ya recomendados
        juegos_recomendados = []
        for _ in range(muestra_aleatoria):
            juegos_no_recomendados = chunk_tags_and_genres[~chunk_tags_and_genres['title'].isin(juegos_recomendados)]

            if juegos_no_recomendados.empty:
                break

            juego_a_recomendar = juegos_no_recomendados.sample(n=1)
            juegos_recomendados.append(juego_a_recomendar['title'].iloc[0])

            # Puedes imprimir los juegos recomendados en cada ciclo para verificar
            print(juegos_recomendados)

        if juegos_recomendados:
            return {"recomendaciones": juegos_recomendados}
        else:
            return {"message": "No quedan juegos no recomendados."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}


In [65]:
def recomendacion_juego_lim3_1(id_juego: int):
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_combinado2[df_combinado2['item_id'] == id_juego]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

        # Obtener la primera fila de juego_seleccionado
        primera_fila = juego_seleccionado.iloc[[0]]

        title_game_and_genres = ' '.join(primera_fila['title'].fillna('').astype(str) + ' ' + primera_fila['genres'].fillna('').astype(str))

        tfidf_vectorizer = TfidfVectorizer()

        # Obtener las columnas 'title' y 'genres' antes de la concatenación
        chunk_tags_and_genres = df_combinado2[['title', 'genres']].fillna('').astype(str)

        # Agregar una nueva columna 'concatenated'
        chunk_tags_and_genres['concatenated'] = chunk_tags_and_genres['title'] + ' ' + chunk_tags_and_genres['genres']

        # Filtrar juegos duplicados antes de tomar una muestra aleatoria
        chunk_tags_and_genres = chunk_tags_and_genres.drop_duplicates(subset=['concatenated'])

        # Seleccionar juegos no recomendados aleatorios sin reemplazo
        juegos_recomendados = []
        for _ in range(5):
            juegos_no_recomendados = chunk_tags_and_genres[~chunk_tags_and_genres['title'].isin(juegos_recomendados)]

            if juegos_no_recomendados.empty:
                break

            juego_a_recomendar = np.random.choice(juegos_no_recomendados['title'].values, size=1, replace=False)[0]
            juegos_recomendados.append(juego_a_recomendar)

        if juegos_recomendados:
            return {"recomendaciones": juegos_recomendados}
        else:
            return {"message": "No quedan juegos no recomendados."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}


In [66]:
a = recomendacion_juego_lim3_1(10)
a

{'recomendaciones': ['NecroVisioN: Lost Company',
  'Project Genom',
  'Beyond Flesh and Blood',
  'Retro Game Crunch',
  'Lili: Child of Geos - Complete Edition']}

-----
-----
----
-----
# CODIGO A USAR EN LA FUNCION 6!!


#RECOMENDACION DE JUEGO CON ENTRADA DE JUEGO

In [2]:
# df_combinado2 = pd.read_csv("df_funciones/df_recomend_juego_fc6.csv")

In [10]:
df_combinado2 = pd.read_parquet("df_funciones/df_recomed_juego_reducido.parquet")

# VERSIÓN FINAL

In [11]:
def recomendacion_juego_lim3_2(id_juego: int):
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_combinado2[df_combinado2['item_id'] == id_juego]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

        # Obtener la primera fila de juego_seleccionado
        primera_fila = juego_seleccionado.iloc[[0]]

        title_game_and_genres = ' '.join(primera_fila['title'].fillna('').astype(str) + ' ' + primera_fila['genres'].fillna('').astype(str))

        tfidf_vectorizer = TfidfVectorizer()

        # Obtener las columnas 'title' y 'genres' antes de la concatenación
        chunk_tags_and_genres = df_combinado2[['title', 'genres']].fillna('').astype(str)

        # Agregar una nueva columna 'concatenated'
        chunk_tags_and_genres['concatenated'] = chunk_tags_and_genres['title'] + ' ' + chunk_tags_and_genres['genres']

        # Filtrar juegos duplicados antes de tomar una muestra aleatoria
        chunk_tags_and_genres = chunk_tags_and_genres.drop_duplicates(subset=['concatenated'])

        muestra_aleatoria = 25000

        # Tomar una muestra aleatoria del conjunto de datos, excluyendo los juegos ya recomendados
        juegos_recomendados = []
        while len(juegos_recomendados) < 5:
            juegos_no_recomendados = chunk_tags_and_genres[~chunk_tags_and_genres['title'].isin(juegos_recomendados)]

            if juegos_no_recomendados.empty:
                break

            juego_a_recomendar = juegos_no_recomendados.sample(n=1)
            juegos_recomendados.append(juego_a_recomendar['title'].iloc[0])

            
        if juegos_recomendados:
            return {"recomendaciones": juegos_recomendados}
        else:
            return {"message": "No quedan juegos no recomendados."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}


-----

PRUEBA DE CODIGO

In [12]:
res = recomendacion_juego_lim3_2(4000)
res

{'recomendaciones': ['Carp Fishing Simulator',
  'Vitrum',
  'TownCraft',
  'Spellbind',
  'Super Chain Crusher Horizon']}

In [13]:
a = recomendacion_juego_lim3_2(282010)
a

{'recomendaciones': ['The Haunted: Hells Reach',
  'The Defenders: The Second Wave',
  "Eternity's Child",
  'Serious Sam Classic: The Second Encounter',
  'Fallout Tactics: Brotherhood of Steel']}

In [14]:
b = recomendacion_juego_lim3_2(20900)
b

{'recomendaciones': ['Homefront',
  'Sim Junta',
  "Letter Quest: Grimm's Journey",
  'The Longest Journey',
  'BRAWL']}

In [15]:
c = recomendacion_juego_lim3_2(70400)
c

{'recomendaciones': ['Sid Meier’s Ace Patrol',
  'Catacomb Kids',
  'Puzzle Agent 2',
  'Rayman® Origins',
  'APOX']}

In [16]:
d = recomendacion_juego_lim3_2(42700)
d

{'recomendaciones': ['Indiana Jones® and the Fate of Atlantis™',
  'POSTAL',
  'Villagers and Heroes',
  'Echo of Soul',
  'Enigmatis: The Ghosts of Maple Creek']}

In [17]:
e = recomendacion_juego_lim3_2(63710)
e

{'recomendaciones': ['BattleBlock Theater®',
  'Puzzle Kingdoms',
  'Not The Robots',
  'Biglands: A Game Made By Kids',
  'Strife®']}

-----
---
-----
----



----

LISTADO DE JUEGOS PARA PROBAR LA FUNCION

In [12]:
listado_juegos.head(50)

,item_id,item_name
0,10.0,Counter-Strike
1,20.0,Team Fortress Classic
2,30.0,Day of Defeat
3,40.0,Deathmatch Classic
4,50.0,Half-Life: Opposing Force
5,60.0,Ricochet
6,70.0,Half-Life
7,130.0,Half-Life: Blue Shift
8,300.0,Day of Defeat: Source
9,240.0,Counter-Strike: Source


----
----
OTRA FORMA DE MISMO CODIGO
---
---

In [63]:
def recomendacion_juego_lim3_2(id_juego: int):
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_combinado2[df_combinado2['item_id'] == id_juego]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

        # Obtener la primera fila de juego_seleccionado
        primera_fila = juego_seleccionado.iloc[[0]]

        title_game_and_genres = ' '.join(primera_fila['title'].fillna('').astype(str) + ' ' + primera_fila['genres'].fillna('').astype(str))

        tfidf_vectorizer = TfidfVectorizer()

        # Obtener las columnas 'title' y 'genres' antes de la concatenación
        chunk_tags_and_genres = df_combinado2[['title', 'genres']].fillna('').astype(str)

        # Agregar una nueva columna 'concatenated'
        chunk_tags_and_genres['concatenated'] = chunk_tags_and_genres['title'] + ' ' + chunk_tags_and_genres['genres']

        # Filtrar juegos duplicados antes de tomar una muestra aleatoria
        chunk_tags_and_genres = chunk_tags_and_genres.drop_duplicates(subset=['concatenated'])

        muestra_aleatoria = 25000

        # Tomar una muestra aleatoria del conjunto de datos, excluyendo los juegos ya recomendados
        juegos_recomendados = []
        while len(juegos_recomendados) < 5:
            juegos_no_recomendados = chunk_tags_and_genres[~chunk_tags_and_genres['title'].isin(juegos_recomendados)]

            if juegos_no_recomendados.empty:
                break

            juego_a_recomendar = juegos_no_recomendados.sample(n=1)
            juegos_recomendados.append(juego_a_recomendar['title'].iloc[0])

            
        if juegos_recomendados:
            return {"recomendaciones": juegos_recomendados}
        else:
            return {"message": "No quedan juegos no recomendados."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}

In [64]:
a_2 = recomendacion_juego_lim3_2(10)
a_2

{'recomendaciones': ['East Tower - Kurenai',
  'The Return Home',
  'CounterAttack',
  'Critter Crunch',
  'Streamline']}

----
----
----
----
----


MODIFICACIONES INCIALES

-----
----


Funcion inicial modificada!


In [ ]:
# @app.get('/RecomendacionJuego/{id_juego}')
def recomendacion_juego(id_juego: int,df_entrada = df_combinado2_filtrado ):
    '''
    Endpoint para obtener una lista de juegos recomendados similares a un juego dado.

    Parámetros:
    - id_juego (int): ID del juego para el cual se desean obtener recomendaciones.

    Respuestas:
    - 200 OK: Retorna una lista con 5 juegos recomendados similares al juego ingresado.
    - 404 Not Found: Si no se encuentra el juego con el ID especificado.
    - 500 Internal Server Error: En caso de cualquier otro error, proporciona detalles de la excepción.

    Ejemplo de Uso:
    - /RecomendarJuego/123

    Ejemplo de Respuesta Exitosa:
    [
        {"title": "Juego A"},
        {"title": "Juego B"},
        {"title": "Juego C"},
        {"title": "Juego D"},
        {"title": "Juego E"}
    ]
    '''
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_entrada[df_entrada['item_id'] == id_juego]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

        # Genera una variable con el titulo y genero en tipo str
        title_game_and_genres = ' '.join(juego_seleccionado['title'].fillna('').astype(str) + ' ' + juego_seleccionado['genres'].fillna('').astype(str))
        
        # Genera una cadena de texto con los titulos y los generos
        chunk_tags_and_genres = df_entrada['title'].fillna('').astype(str) + ' ' + df_entrada['genres'].fillna('').astype(str)
        
        # Genera una lista con dos elementos, el juego cargado y el resto de los juegos a comparar
        games_to_compare = [title_game_and_genres] + chunk_tags_and_genres.tolist()

        tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9)
        tfidf_matrix = tfidf_vectorizer.fit_transform(games_to_compare)

        # Generamos una limitacion truncando datos por un gran exceso de datos
        # Selecciona las primeras 15000 características
        tfidf_matrix_subset = tfidf_matrix[:15000, :]

        # Ahora a esos datos truncados toma los mejores 1000 campos y los compara con el dato de entrada
        svd = TruncatedSVD(n_components=1000)
        tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix_subset)

        # Calcula la similiridad de cosine
        # Calcula la matriz de similitud entre juego de entrada y resto de los juegos
        similarity_scores = cosine_similarity(tfidf_matrix_reduced)

        if similarity_scores is not None:
            # Ordena los indices similares de forma descendente y se toma desde el segundo para tomar los mas similares
            similar_games_indices = similarity_scores[0].argsort()[::-1]

            # Tomar las mejores 5 recomendaciones sin incluir el juego seleccionado
            num_recommendations = 5
            recommended_games = df_entrada.loc[similar_games_indices[1:num_recommendations + 1]]
            unique_recommendations = set()
            filtered_recommendations = []

            for index in similar_games_indices[1:]:
                game = df_entrada.loc[index, 'title']
                if game not in unique_recommendations:
                    unique_recommendations.add(game)
                    filtered_recommendations.append(game)

                if len(filtered_recommendations) >= num_recommendations:
                    break

            # Devolver la lista de juegos recomendados
            recommended_games = [{"title": game} for game in filtered_recommendations]

            return recommended_games

        return {"message": "No se encontraron juegos similares."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}

------
----
----


In [5]:
id_juego = 10

In [7]:
# Busca el juego en el DataFrame por ID
juego_seleccionado = df_combinado2[df_combinado2['item_id'] == id_juego]

In [6]:


# Verifica si el juego con el ID especificado existe
if juego_seleccionado.empty:
    raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

# Obtener la primera fila de juego_seleccionado
primera_fila = juego_seleccionado.iloc[[0]]

title_game_and_genres = ' '.join(primera_fila['title'].fillna('').astype(str) + ' ' + primera_fila['genres'].fillna('').astype(str))

tfidf_vectorizer = TfidfVectorizer()

# Obtener las columnas 'title' y 'genres' antes de la concatenación
chunk_tags_and_genres = df_combinado2[['title', 'genres']].fillna('').astype(str)

# Agregar una nueva columna 'concatenated'
chunk_tags_and_genres['concatenated'] = chunk_tags_and_genres['title'] + ' ' + chunk_tags_and_genres['genres']

# Filtrar juegos duplicados antes de tomar una muestra aleatoria
chunk_tags_and_genres = chunk_tags_and_genres.drop_duplicates(subset=['concatenated'])

muestra_aleatoria = 25000

# Tomar una muestra aleatoria del conjunto de datos, excluyendo los juegos ya recomendados
juegos_recomendados = []
while len(juegos_recomendados) < 5:
    juegos_no_recomendados = chunk_tags_and_genres[~chunk_tags_and_genres['title'].isin(juegos_recomendados)]

    if juegos_no_recomendados.empty:
        break

    juego_a_recomendar = juegos_no_recomendados.sample(n=1)
    juegos_recomendados.append(juego_a_recomendar['title'].iloc[0])

    
if juegos_recomendados:
    return {"recomendaciones": juegos_recomendados}
else:
    return {"message": "No quedan juegos no recomendados."}



HTTPException: 